In [1]:
import torch
import numpy as np
from Traj2Dataset import TrajDataset, DatasetTransform
from torch.utils.data import DataLoader
from pytorch_lightning import seed_everything
from model import LitMLP

In [2]:
root_dir = 'dataset'
system = 'great-piquant-bumblebee'
SEED = 42
num_workers = 0
seed_everything(SEED)

Global seed set to 42


42

In [3]:
state_norms, action_norms = TrajDataset.norms(system)

state_dim = len(state_norms)
action_dim = len(action_norms)

mean = state_norms[:, 0]  # mean
std = state_norms[:, 1]  # std_dev
transform = DatasetTransform(mean, std)

target_mean = action_norms[:, 0]  # mean
target_std = action_norms[:, 1]  # std_dev
target_transform = DatasetTransform(target_mean, target_std)

test_dataset = TrajDataset(system, root_dir, train=False,
                           transform=transform,
                           target_transform=target_transform)


In [4]:
path = "./submission/models/great-piquant-bumblebee-v1.ckpt"
model = LitMLP.load_from_checkpoint(
            path, in_dims=state_dim, out_dims=action_dim)

In [5]:
def upscale(x):
    return x * torch.Tensor(target_std) + torch.Tensor(target_mean)


test_dataloader = DataLoader(test_dataset,
                             batch_size=1, shuffle=False,
                             num_workers=0)

for i, (x, y) in enumerate(test_dataloader):
    print(f'Time-step: {i}')
    print(f'Target: {y}')
    print(f'Prediction: {model(x)}')

Time-step: 0
Target: tensor([[-8.0874, -6.0669]])
Prediction: tensor([[-2.5723, -1.0713]], grad_fn=<AddmmBackward>)
Time-step: 1
Target: tensor([[-7.0414, -5.2822]])
Prediction: tensor([[-9.2329, -5.3372]], grad_fn=<AddmmBackward>)
Time-step: 2
Target: tensor([[-6.1049, -4.5798]])
Prediction: tensor([[-6.9179, -5.4170]], grad_fn=<AddmmBackward>)
Time-step: 3
Target: tensor([[-5.2674, -3.9515]])
Prediction: tensor([[-6.4295, -5.2283]], grad_fn=<AddmmBackward>)
Time-step: 4
Target: tensor([[-4.5189, -3.3900]])
Prediction: tensor([[-6.1338, -4.7486]], grad_fn=<AddmmBackward>)
Time-step: 5
Target: tensor([[-3.8506, -2.8886]])
Prediction: tensor([[-5.7356, -4.2539]], grad_fn=<AddmmBackward>)
Time-step: 6
Target: tensor([[-3.2543, -2.4413]])
Prediction: tensor([[-5.4069, -3.8989]], grad_fn=<AddmmBackward>)
Time-step: 7
Target: tensor([[-2.7229, -2.0427]])
Prediction: tensor([[-5.2611, -3.6715]], grad_fn=<AddmmBackward>)
Time-step: 8
Target: tensor([[-2.2501, -1.6880]])
Prediction: tensor([[-